## 1. Извлечение ключевых фраз из текстов
- **a. Классический метод**  
    Использован алгоритм:  
    - YAKE  
- **b. Метод с использованием языковой модели (LLM)**  
    Была использована модель:
    - Google Gemini 2.5 Flash Lite по API.

---

## 2. Построение системы поиска

- **Вариант 1:** Поиск на основе TF-IDF
- **Вариант 2:** Поиск на основе эмбеддингов

### Функция поиска
- **Вход:** текстовый запрос  
- **Выход:** топ-5 релевантных аннотаций

---

## 3. Сравнение подходов

- Примеры извлечённых ключевых фраз для каждого метода
- Сравнение результатов поиска по двум вариантам


> Данная книга является только предоставлением результатов (итоговым отчетом), основные этапы и полный код можно найти в последующих ноутбуках по нумерации

# Часть 1: Извлечение ключевых фраз (Задача 1)

**Классический метод (YAKE)**
YAKE анализирует текст, выявляя слова и фразы, которые статистически выделяются на фоне остального текста. Находит наиболее значимые фразы в тексте, которые отражают его основное содержание.

* **Метод с использованием LLM.**
Метод основанный на обращении к большой языковой модели подразумевает более качественный вывод ключевых слов, благодаря способности модели обобщать и улавливать семантический смысл текстов.
* **Методы реализации** 
Реализация этого метода была осуществлена с помощью обращения к Google Gemini Flash 2.5 Lite по API. Она является быстрой, дешевой и соотвествует обработки больших корпусов текста, что прямо упомянуто в документации к модели. Дополнительное преимущество: Structured Output позволяют сразу распарсить ответ модели и не тратить лишние токены на просьбы вывести в json структуре с примерами.
Ввиду ограничений в количестве запросов во free tier плане гугл и размере данных было приянто решение сократить выборку до 200 случайных ед аннотаций, однако реализованные методы являются масштабируемыми для любого количества аннотаций.

In [1]:
import os
import sys
from dotenv import load_dotenv
load_dotenv()

import pandas as pd

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from src.utils import load_dataset
df = load_dataset("df_with_llmkeyphrases")
df_keys = df[["abstract", "yake_keywords", "keyphrases"]]

Данные успешно загружены из ../data/df_with_llmkeyphrases.parquet. Количество записей: 200


In [ ]:
df_keys = pd.read_csv("../data/selected_examples.csv")
df_keys.head()

,abstract,yake_keywords,keyphrases
0,Представлен усовершенствованный метод парных с...,['котором посредством табличных' 'посредством ...,['усовершенствованный метод парных сравнений' ...
1,Статья посвящена разработке комплексной модели...,['Статья посвящена разработке' 'посвящена разр...,['комплексная модель диктора' 'текстонезависим...
2,В работе рассмотрена обобщенная модель образов...,['образования новой фазы' 'переходе первого ро...,['образование новой фазы' 'фазовый переход пер...
3,Предложено обобщение класса экспоненциально-ст...,['обобщенных распределений Лапласа' 'Предложен...,['экспоненциально-степенные распределения'\n '...
4,Рассматривается задача объединения графов с об...,['Petri Nets Tools' 'Colored Petri Nets' 'паке...,['Объединение графов' 'Сети Петри' 'Общая част...


keyphrases - словосочетания извлеченные LLM

Я отобрал примеры, я вывел их в отдельный selected_samples файл, так как выборка из 200 абстрактов выполняется случайно, я не стал ставить конкретные сиды на генерацию и на отбор. Тем не менее выбранные показатели довольно легко находятся при беглом просмотре с любой конфигурацией, ведь паттерн одинаков, как и соотвествующие выовды:

1) +LLM Семантика. LLM выдает чистые и осмысленные концепции, в то время как YAKE давал много артефактов, включая академический шум в виде "в статье рассматрвиается"
2) +LLM LLM хорошо обобщает и находит релевантные ключевые фразы. Статистический метод на это не способен
3) +Yake YAKE! все таки смог найти некоторые четкие уникальные термины, это видно в последнем пятом примере (Petri Nets Tools), однако эти уникальные термины могут быть лишь просто уникальным упоминанием, а не действительно ключевым словом.
4) +YAKE Статистический метод справляется с 100 и более аннотациями в секунду и стоит практически ничего в денежном эквиваленте, в то время как использование LLM при моих параметрах батчинга 20 и 10 одновременных запросах способен только на примерно 10 аннотаций в секунду
5) +YAKE реализация этого метода довольно проста, в то время как для LLM нужно построить небольшой пайплайн. 

| Критерий | YAKE (Классический подход) | LLM (Gemini 2.5 Flash Lite) |
| :--- | :--- | :--- |
| **Качество фраз** | Низкое-Среднее (извлекает n-граммы, шум, "рваные" фразы) | Высокое (извлекает семантические концепции, обобщает) |
| **Скорость** | Очень высокая (<1с на ~200 док.) | Низкая (20с на 200 док.) |
| **Стоимость** | 0 (бесплатно) | Низкая, но не нулевая (~$0.01 за 200 док.) |
| **Сложность реализации** | Низкая (импорт библиотеки) | Средняя (требует инженерии API, обработки ошибок) |
| **Сценарий использования**| Быстрый baseline, поиск по точным терминам, прототипирование. | Production-системы, семантический анализ, задачи, где качество важнее скорости. |

Выбор метода зависит от задачи: для быстрого прототипа и простого извлечения терминов YAKE является адекватным baseline, но для построения качественной системы, понимающей смысл текста, LLM-подход является одним из лучших

# Часть 2: Семантический поиск

* Оба метода предстовляют собой векторный поиск, разниц лишь в том как векторы создаются.
### **TF-IDF**
Насколько важным является каждое конкретное слово из предопределенного словаря для данного текста. Это **явное (explicit)** представление, где каждая размерность вектора соответствует одному слову. СОздается путем подсчета статистики слов по всему корпусу, вектор зависит от остальных документов в коллекции.
### **Эмбиддинги**
Векторы текстов создаются трансформером предобученной на больших объемах текста чтобы понимать контекст и семантические связи между словами. Вектор для одного документа **не зависит от других документов в твоей коллекции** - он вычисляется только на основе самого текста и знаний модели

In [4]:
from src.search_tf_idf.search import TfidfSearch
from src.search_embeddings.engine import EmbeddingSearchEngine
from src.classic_keywords.preprocessing import preprocess_text, lemmatize_word
from IPython.display import display, Markdown
import re

In [5]:
print("Инициализация TF-IDF...")
tfidf_search_engine = TfidfSearch()
tfidf_search_engine.build_index(
    lemmatized_texts=df['lemmatized_abstract'],
    original_texts=df['abstract']
)

Инициализация TF-IDF...


KeyError: 'lemmatized_abstract'

In [ ]:
print("\nИнициализация Embedding Search...")
embedding_search_engine = EmbeddingSearchEngine()
embedding_search_engine.build_index(df['abstract'])

In [ ]:
def highlight_keywords(text: str, query: str, preprocessor_func) -> str:
    query_lemmas = set(preprocessor_func(query).split())
    tokens = re.split(r'(\s+)', text) 
    highlighted_tokens = []
    for token in tokens:
        if not token.strip():
            highlighted_tokens.append(token)
            continue
        cleaned_token = token.strip('.,!?:;()[]{}')
        if lemmatize_word(cleaned_token.lower()) in query_lemmas:
            highlighted_tokens.append(f"**{token}**")
        else:
            highlighted_tokens.append(token)
    return "".join(highlighted_tokens)

print("\nПодготовка завершена. Все поисковые системы готовы к работе.")

In [ ]:
query1 = "высокотехнологичная медицинская помощь сердечно-сосудистая хирургия ДВФО"

tfidf_results1 = tfidf_search_engine.search(query1, preprocessor_func=preprocess_text)
embedding_results1 = embedding_search_engine.search(query1, top_n=5)

tfidf_md = ""
for i, (doc_index, doc_text) in enumerate(tfidf_results1, 1):
    highlighted = highlight_keywords(doc_text, query1, preprocess_text)
    tfidf_md += f"**{i}. Индекс {doc_index}** <br> `{highlighted[:250].replace('*', '')}...` <br><br>"

embed_md = ""
for i, (doc_index, doc_text, score) in enumerate(embedding_results1, 1):
    embed_md += f"**{i}. Индекс {doc_index} (Score: {score:.3f})** <br> `{doc_text[:250]}...` <br><br>"

comparison_table = f"""
| TF-IDF (Лексический поиск) | Семантический поиск (FRIDA) |
| :--- | :--- |
| {tfidf_md} | {embed_md} |

Сравним результаты: запрос был выбран специально с включением ключевых слов и легко можно заметить оба поисковых метода смогли верно определить таргет аннотацию. В целом такой результат и ожидался.
> Ключевой момент: поиск TF занял **0.033547 секунд**, в то время как эмбиддинг справился за **1.29 секунд**. 
Это показывает разницу: статистический метод молниеносен, он не нагружает систему и не требует больших вычислений.

In [ ]:
query1 = "кардиохирургия ВМП Дальний Восток"

tfidf_results1 = tfidf_search_engine.search(query1, preprocessor_func=preprocess_text)
embedding_results1 = embedding_search_engine.search(query1, top_n=5)

tfidf_md = ""
for i, (doc_index, doc_text) in enumerate(tfidf_results1, 1):
    highlighted = highlight_keywords(doc_text, query1, preprocess_text)
    tfidf_md += f"**{i}. Индекс {doc_index}** <br> `{highlighted[:250].replace('*', '')}...` <br><br>"

embed_md = ""
for i, (doc_index, doc_text, score) in enumerate(embedding_results1, 1):
    embed_md += f"**{i}. Индекс {doc_index} (Score: {score:.3f})** <br> `{doc_text[:250]}...` <br><br>"

comparison_table = f"""
| TF-IDF (Лексический поиск) | Семантический поиск (FRIDA) |
| :--- | :--- |
| {tfidf_md} | {embed_md} |
"""

display(Markdown(comparison_table))

Это пример нового поискового запроса с целью на ту же самую аннотацию, однако теперь запрос был переформулирован синонимично, с сохранением смысла.
> Легко заметить что эмбиддинг справился, в то время как лексический поиск полностью провалился не предоставив аннотацию даже в пятерке. 

Здесь и предоставляется снова нюанс: предобученные модели качественно справляются со своей задачей часто гораздо лучше чем любой другой статистический метод основанный на грубых вычислениях, однако они входят в компромис с вычислениями, история похожая на сравнения извлечения ключевых слов
Кроме того, вычисление эмбиддингов тоже требует вычислений, для хорошего результата с использованием лучших моделей время сильно зависит от мощностей.